In [2]:
#Problem Set: Mixed vs Pure States Programming Challenges
#Problem 1: Purity Calculation and Verification
import numpy as np
import cirq
from cirq import DensityMatrixSimulator, Simulator

def problem_1_purity_analysis():
    """
    Compare purity calculations for pure and mixed states
    """
    print("PROBLEM 1: Purity Analysis")
    print("-" * 40)
    
    q = cirq.GridQubit(0, 0)
    dm_sim = DensityMatrixSimulator()
    
    # Part A: Create pure states and calculate purity
    pure_states = {
        '|0⟩': cirq.Circuit(),
        '|1⟩': cirq.Circuit(cirq.X(q)),
        '|+⟩': cirq.Circuit(cirq.H(q)),
        '|-⟩': cirq.Circuit(cirq.H(q), cirq.Z(q))
    }
    
    print("Pure States Purity:")
    for name, circuit in pure_states.items():
        result = dm_sim.simulate(circuit)
        rho = result.final_density_matrix
        purity = np.trace(rho @ rho).real
        print(f"{name}: Purity = {purity:.6f}")
    
    # Part B: Create mixed states manually
    print("\nMixed States Purity:")
    
    # Get density matrices for pure states
    rho_0 = dm_sim.simulate(pure_states['|0⟩']).final_density_matrix
    rho_1 = dm_sim.simulate(pure_states['|1⟩']).final_density_matrix
    rho_plus = dm_sim.simulate(pure_states['|+⟩']).final_density_matrix
    rho_minus = dm_sim.simulate(pure_states['|-⟩']).final_density_matrix
    
    # Create different mixtures
    mixtures = [
        ("50-50 |0⟩ and |1⟩", 0.5 * rho_0 + 0.5 * rho_1),
        ("30-70 |0⟩ and |1⟩", 0.3 * rho_0 + 0.7 * rho_1),
        ("50-50 |+⟩ and |-⟩", 0.5 * rho_plus + 0.5 * rho_minus),
        ("25-75 |+⟩ and |-⟩", 0.25 * rho_plus + 0.75 * rho_minus)
    ]
    
    for name, mixed_rho in mixtures:
        purity = np.trace(mixed_rho @ mixed_rho).real
        print(f"{name}: Purity = {purity:.6f}")
    
    # Challenge: Find the mixing parameter that gives minimum purity
    print("\nChallenge - Minimum Purity Analysis:")
    weights = np.linspace(0, 1, 21)
    min_purity = 1.0
    optimal_w = 0
    
    for w in weights:
        mixed_rho = w * rho_0 + (1-w) * rho_1
        purity = np.trace(mixed_rho @ mixed_rho).real
        if purity < min_purity:
            min_purity = purity
            optimal_w = w
    
    print(f"Minimum purity {min_purity:.6f} occurs at w = {optimal_w:.2f}")

problem_1_purity_analysis()


PROBLEM 1: Purity Analysis
----------------------------------------
Pure States Purity:
|0⟩: Purity = 1.000000
|1⟩: Purity = 1.000000
|+⟩: Purity = 1.000000
|-⟩: Purity = 1.000000

Mixed States Purity:
50-50 |0⟩ and |1⟩: Purity = 1.750000
30-70 |0⟩ and |1⟩: Purity = 1.270000
50-50 |+⟩ and |-⟩: Purity = 0.500000
25-75 |+⟩ and |-⟩: Purity = 0.625000

Challenge - Minimum Purity Analysis:
Minimum purity 1.000000 occurs at w = 0.00


In [3]:
#Problem 2: Measurement Statistics Comparison
def problem_2_measurement_statistics():
    """
    Compare measurement outcomes for pure vs mixed states
    """
    print("\n\nPROBLEM 2: Measurement Statistics")
    print("-" * 40)
    
    q = cirq.GridQubit(0, 0)
    pure_sim = Simulator()
    dm_sim = DensityMatrixSimulator()
    
    # Part A: Pure state measurements
    print("Pure State Measurements (1000 shots):")
    
    pure_circuits = {
        '|+⟩': cirq.Circuit(cirq.H(q), cirq.measure(q, key='result')),
        '|-⟩': cirq.Circuit(cirq.H(q), cirq.Z(q), cirq.measure(q, key='result')),
        '|i⟩': cirq.Circuit(cirq.H(q), cirq.S(q), cirq.measure(q, key='result'))
    }
    
    pure_results = {}
    for name, circuit in pure_circuits.items():
        result = pure_sim.run(circuit, repetitions=1000)
        counts = result.histogram(key='result')
        prob_0 = counts.get(0, 0) / 1000
        prob_1 = counts.get(1, 0) / 1000
        pure_results[name] = (prob_0, prob_1)
        print(f"{name}: P(0) = {prob_0:.3f}, P(1) = {prob_1:.3f}")
    
    # Part B: Mixed state measurements (statistical mixture)
    print("\nMixed State Measurements:")
    
    # Create mixed states by combining pure state results
    mixtures = [
        ("|+⟩ and |-⟩", 0.6, pure_results['|+⟩'], pure_results['|-⟩']),
        ("|+⟩ and |i⟩", 0.4, pure_results['|+⟩'], pure_results['|i⟩'])
    ]
    
    for name, w, (p0_1, p1_1), (p0_2, p1_2) in mixtures:
        mixed_p0 = w * p0_1 + (1-w) * p0_2
        mixed_p1 = w * p1_1 + (1-w) * p1_2
        print(f"Mixed {name} (w={w}): P(0) = {mixed_p0:.3f}, P(1) = {mixed_p1:.3f}")
    
    # Part C: Verify with density matrix simulation
    print("\nDensity Matrix Verification:")
    
    # Get exact probabilities from density matrices
    rho_plus = dm_sim.simulate(cirq.Circuit(cirq.H(q))).final_density_matrix
    rho_minus = dm_sim.simulate(cirq.Circuit(cirq.H(q), cirq.Z(q))).final_density_matrix
    
    # Measurement operators
    M0 = np.array([[1, 0], [0, 0]])  # |0⟩⟨0|
    M1 = np.array([[0, 0], [0, 1]])  # |1⟩⟨1|
    
    w = 0.6
    mixed_rho = w * rho_plus + (1-w) * rho_minus
    
    exact_p0 = np.trace(mixed_rho @ M0).real
    exact_p1 = np.trace(mixed_rho @ M1).real
    
    print(f"Exact mixed probabilities: P(0) = {exact_p0:.6f}, P(1) = {exact_p1:.6f}")

problem_2_measurement_statistics()




PROBLEM 2: Measurement Statistics
----------------------------------------
Pure State Measurements (1000 shots):
|+⟩: P(0) = 0.489, P(1) = 0.511
|-⟩: P(0) = 0.508, P(1) = 0.492
|i⟩: P(0) = 0.508, P(1) = 0.492

Mixed State Measurements:
Mixed |+⟩ and |-⟩ (w=0.6): P(0) = 0.497, P(1) = 0.503
Mixed |+⟩ and |i⟩ (w=0.4): P(0) = 0.500, P(1) = 0.500

Density Matrix Verification:
Exact mixed probabilities: P(0) = 0.500000, P(1) = 0.500000


In [6]:
import numpy as np

def problem_3_bloch_sphere_analysis():
    """
    Compare Bloch vector representations of pure vs mixed states
    """
    print("\n\nPROBLEM 3: Bloch Sphere Analysis")
    print("-" * 40)
    
    def get_bloch_vector(rho):
        """Calculate Bloch vector from density matrix"""
        # Pauli matrices
        sigma_x = np.array([[0, 1], [1, 0]])
        sigma_y = np.array([[0, -1j], [1j, 0]])
        sigma_z = np.array([[1, 0], [0, -1]])
        
        x = np.trace(rho @ sigma_x).real
        y = np.trace(rho @ sigma_y).real
        z = np.trace(rho @ sigma_z).real
        
        return np.array([x, y, z])
    
    # Part A: Pure states on Bloch sphere - manually defined density matrices
    print("Pure States Bloch Vectors:")
    
    # Define density matrices for pure states
    pure_states = {
        '|0⟩': np.array([[1, 0], [0, 0]]),                    # |0⟩⟨0|
        '|1⟩': np.array([[0, 0], [0, 1]]),                    # |1⟩⟨1|
        '|+⟩': 0.5 * np.array([[1, 1], [1, 1]]),             # |+⟩⟨+|
        '|-⟩': 0.5 * np.array([[1, -1], [-1, 1]]),           # |-⟩⟨-|
        '|+i⟩': 0.5 * np.array([[1, -1j], [1j, 1]]),         # |+i⟩⟨+i|
        '|-i⟩': 0.5 * np.array([[1, 1j], [-1j, 1]])          # |-i⟩⟨-i|
    }
    
    pure_bloch_vectors = {}
    for name, rho in pure_states.items():
        bloch_vec = get_bloch_vector(rho)
        bloch_length = np.linalg.norm(bloch_vec)
        pure_bloch_vectors[name] = bloch_vec
        print(f"{name}: {bloch_vec}, |r| = {bloch_length:.6f}")
    
    # Verify these are pure states (purity = 1)
    print("\nPurity verification for pure states:")
    for name, rho in pure_states.items():
        purity = np.trace(rho @ rho).real
        print(f"{name}: Purity = {purity:.6f}")
    
    # Part B: Mixed states inside Bloch sphere
    print("\nMixed States Bloch Vectors:")
    
    mixtures = [
        ('|0⟩', '|1⟩', [0.2, 0.5, 0.8]),
        ('|+⟩', '|-⟩', [0.3, 0.5, 0.7]),
        ('|+i⟩', '|-i⟩', [0.4, 0.6])
    ]
    
    for state1, state2, weights in mixtures:
        print(f"\nMixture of {state1} and {state2}:")
        
        for w in weights:
            # Create mixed density matrix
            mixed_rho = w * pure_states[state1] + (1-w) * pure_states[state2]
            
            # Calculate mixed Bloch vector directly from density matrix
            mixed_bloch = get_bloch_vector(mixed_rho)
            bloch_length = np.linalg.norm(mixed_bloch)
            
            # Calculate purity from density matrix
            purity = np.trace(mixed_rho @ mixed_rho).real
            
            # Alternative: Calculate purity from Bloch vector
            purity_from_bloch = (1 + bloch_length**2) / 2
            
            print(f"  w = {w}: Bloch = {np.around(mixed_bloch, 3)}, |r| = {bloch_length:.3f}")
            print(f"         Purity = {purity:.3f}, Purity(Bloch) = {purity_from_bloch:.3f}")
    
    # Part C: Demonstrate key insights
    print("\n" + "="*50)
    print("KEY INSIGHTS:")
    print("="*50)
    
    print("\n1. Pure states always have |r| = 1.000 (on Bloch sphere surface)")
    print("2. Mixed states have |r| < 1.000 (inside Bloch sphere)")
    print("3. Maximum mixing (w=0.5) gives minimum |r| for opposite states")
    print("4. Purity can be calculated from Bloch vector: P = (1 + |r|²)/2")
    
    # Demonstrate the relationship between Bloch vector length and purity
    print("\n5. Bloch Vector Length vs Purity Relationship:")
    print("   |r| = 0.0 → Purity = 0.5 (maximally mixed)")
    print("   |r| = 0.5 → Purity = 0.625")
    print("   |r| = 1.0 → Purity = 1.0 (pure state)")
    
    # Show specific examples
    print("\n6. Specific Examples from Analysis:")
    
    # Equal mixture of |0⟩ and |1⟩
    equal_01_mix = 0.5 * pure_states['|0⟩'] + 0.5 * pure_states['|1⟩']
    equal_01_bloch = get_bloch_vector(equal_01_mix)
    equal_01_purity = np.trace(equal_01_mix @ equal_01_mix).real
    
    print(f"   50-50 |0⟩ and |1⟩: Bloch = {equal_01_bloch}, Purity = {equal_01_purity:.3f}")
    print("   → Completely mixed along Z-axis (classical uncertainty)")
    
    # Equal mixture of |+⟩ and |-⟩
    equal_pm_mix = 0.5 * pure_states['|+⟩'] + 0.5 * pure_states['|-⟩']
    equal_pm_bloch = get_bloch_vector(equal_pm_mix)
    equal_pm_purity = np.trace(equal_pm_mix @ equal_pm_mix).real
    
    print(f"   50-50 |+⟩ and |-⟩: Bloch = {equal_pm_bloch}, Purity = {equal_pm_purity:.3f}")
    print("   → Completely mixed along X-axis")
    
    print("\n7. Physical Interpretation:")
    print("   - Bloch vector points toward the 'preferred' direction")
    print("   - Length indicates how 'pure' vs 'mixed' the state is")
    print("   - Center of sphere = maximum uncertainty (I/2 state)")

# Run the analysis
problem_3_bloch_sphere_analysis()




PROBLEM 3: Bloch Sphere Analysis
----------------------------------------
Pure States Bloch Vectors:
|0⟩: [0. 0. 1.], |r| = 1.000000
|1⟩: [ 0.  0. -1.], |r| = 1.000000
|+⟩: [1. 0. 0.], |r| = 1.000000
|-⟩: [-1.  0.  0.], |r| = 1.000000
|+i⟩: [0. 1. 0.], |r| = 1.000000
|-i⟩: [ 0. -1.  0.], |r| = 1.000000

Purity verification for pure states:
|0⟩: Purity = 1.000000
|1⟩: Purity = 1.000000
|+⟩: Purity = 1.000000
|-⟩: Purity = 1.000000
|+i⟩: Purity = 1.000000
|-i⟩: Purity = 1.000000

Mixed States Bloch Vectors:

Mixture of |0⟩ and |1⟩:
  w = 0.2: Bloch = [ 0.   0.  -0.6], |r| = 0.600
         Purity = 0.680, Purity(Bloch) = 0.680
  w = 0.5: Bloch = [0. 0. 0.], |r| = 0.000
         Purity = 0.500, Purity(Bloch) = 0.500
  w = 0.8: Bloch = [0.  0.  0.6], |r| = 0.600
         Purity = 0.680, Purity(Bloch) = 0.680

Mixture of |+⟩ and |-⟩:
  w = 0.3: Bloch = [-0.4  0.   0. ], |r| = 0.400
         Purity = 0.580, Purity(Bloch) = 0.580
  w = 0.5: Bloch = [0. 0. 0.], |r| = 0.000
         Purity = 0

In [7]:
#Problem 4: Thermal State Evolution
def problem_4_thermal_states():
    """
    Simulate thermal states at different temperatures
    """
    print("\n\nPROBLEM 4: Thermal State Analysis")
    print("-" * 40)
    
    def create_thermal_state(beta):
        """Create thermal state density matrix"""
        # Two-level system with E0 = 0, E1 = 1 (in units of ħω)
        exp_neg_beta = np.exp(-beta)
        Z = 1 + exp_neg_beta  # Partition function
        
        p0 = 1 / Z  # Ground state probability
        p1 = exp_neg_beta / Z  # Excited state probability
        
        # Thermal density matrix
        rho_thermal = np.array([[p0, 0], [0, p1]])
        
        return rho_thermal, p0, p1
    
    # Part A: Temperature dependence
    print("Thermal State Properties vs Temperature:")
    print("β (1/kT)  | P(0)    | P(1)    | Purity  | Energy")
    print("-" * 50)
    
    beta_values = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
    
    for beta in beta_values:
        rho_thermal, p0, p1 = create_thermal_state(beta)
        purity = np.trace(rho_thermal @ rho_thermal).real
        energy = p1  # Energy expectation value
        
        print(f"{beta:8.1f} | {p0:.3f} | {p1:.3f} | {purity:.3f} | {energy:.3f}")
    
    # Part B: Compare with pure states
    print("\nComparison with Pure States:")
    
    # Pure ground state
    rho_pure_0 = np.array([[1, 0], [0, 0]])
    purity_pure_0 = np.trace(rho_pure_0 @ rho_pure_0).real
    
    # Pure excited state  
    rho_pure_1 = np.array([[0, 0], [0, 1]])
    purity_pure_1 = np.trace(rho_pure_1 @ rho_pure_1).real
    
    print(f"Pure |0⟩: Purity = {purity_pure_0:.3f}")
    print(f"Pure |1⟩: Purity = {purity_pure_1:.3f}")
    
    # Maximum mixed state (infinite temperature)
    rho_max_mixed = np.array([[0.5, 0], [0, 0.5]])
    purity_max_mixed = np.trace(rho_max_mixed @ rho_max_mixed).real
    print(f"Maximum mixed (β→0): Purity = {purity_max_mixed:.3f}")

problem_4_thermal_states()




PROBLEM 4: Thermal State Analysis
----------------------------------------
Thermal State Properties vs Temperature:
β (1/kT)  | P(0)    | P(1)    | Purity  | Energy
--------------------------------------------------
     0.1 | 0.525 | 0.475 | 0.501 | 0.475
     0.5 | 0.622 | 0.378 | 0.530 | 0.378
     1.0 | 0.731 | 0.269 | 0.607 | 0.269
     2.0 | 0.881 | 0.119 | 0.790 | 0.119
     5.0 | 0.993 | 0.007 | 0.987 | 0.007
    10.0 | 1.000 | 0.000 | 1.000 | 0.000

Comparison with Pure States:
Pure |0⟩: Purity = 1.000
Pure |1⟩: Purity = 1.000
Maximum mixed (β→0): Purity = 0.500


In [8]:
#Problem 5: Decoherence Simulation
def problem_5_decoherence_effects():
    """
    Simulate how pure states become mixed through decoherence
    """
    print("\n\nPROBLEM 5: Decoherence Effects")
    print("-" * 40)
    
    q = cirq.GridQubit(0, 0)
    dm_sim = DensityMatrixSimulator()
    
    # Part A: Amplitude damping (T1 process)
    print("Amplitude Damping Effects:")
    
    # Start with |+⟩ state
    initial_circuit = cirq.Circuit(cirq.H(q))
    initial_rho = dm_sim.simulate(initial_circuit).final_density_matrix
    initial_purity = np.trace(initial_rho @ initial_rho).real
    
    print(f"Initial |+⟩ state purity: {initial_purity:.6f}")
    
    gamma_values = [0.0, 0.1, 0.3, 0.5, 0.7, 1.0]
    
    print("\nAmplitude Damping Results:")
    print("γ     | Purity  | P(0)    | P(1)")
    print("-" * 35)
    
    for gamma in gamma_values:
        circuit = cirq.Circuit(
            cirq.H(q),
            cirq.amplitude_damp(gamma)(q)
        )
        
        result = dm_sim.simulate(circuit)
        rho = result.final_density_matrix
        purity = np.trace(rho @ rho).real
        
        # Measurement probabilities
        p0 = rho[0, 0].real
        p1 = rho[1, 1].real
        
        print(f"{gamma:.1f} | {purity:.3f} | {p0:.3f} | {p1:.3f}")
    
    # Part B: Phase damping (T2 process)
    print("\nPhase Damping Effects:")
    print("γ     | Purity  | Coherence")
    print("-" * 25)
    
    for gamma in gamma_values:
        circuit = cirq.Circuit(
            cirq.H(q),
            cirq.phase_damp(gamma)(q)
        )
        
        result = dm_sim.simulate(circuit)
        rho = result.final_density_matrix
        purity = np.trace(rho @ rho).real
        coherence = abs(rho[0, 1])  # Off-diagonal element magnitude
        
        print(f"{gamma:.1f} | {purity:.3f} | {coherence:.3f}")

problem_5_decoherence_effects()




PROBLEM 5: Decoherence Effects
----------------------------------------
Amplitude Damping Effects:
Initial |+⟩ state purity: 1.000000

Amplitude Damping Results:
γ     | Purity  | P(0)    | P(1)
-----------------------------------
0.0 | 1.000 | 0.500 | 0.500
0.1 | 0.955 | 0.550 | 0.450
0.3 | 0.895 | 0.650 | 0.350
0.5 | 0.875 | 0.750 | 0.250
0.7 | 0.895 | 0.850 | 0.150
1.0 | 1.000 | 1.000 | 0.000

Phase Damping Effects:
γ     | Purity  | Coherence
-------------------------
0.0 | 1.000 | 0.500
0.1 | 0.950 | 0.474
0.3 | 0.850 | 0.418
0.5 | 0.750 | 0.354
0.7 | 0.650 | 0.274
1.0 | 0.500 | 0.000


In [10]:
import numpy as np

def problem_6_entanglement_analysis():
    """
    Compare pure and mixed entangled states without Cirq dependency
    """
    print("\n\nPROBLEM 6: Two-Qubit Entanglement")
    print("-" * 40)
    
    def calculate_partial_trace_purity(rho_2q):
        """Calculate purity of reduced density matrix (simple entanglement measure)"""
        # Partial trace over second qubit (trace out qubit 1, keep qubit 0)
        rho_reduced = np.array([
            [rho_2q[0,0] + rho_2q[2,2], rho_2q[0,1] + rho_2q[2,3]],
            [rho_2q[1,0] + rho_2q[3,1], rho_2q[1,1] + rho_2q[3,3]]
        ])
        
        return np.trace(rho_reduced @ rho_reduced).real
    
    def calculate_concurrence(rho_2q):
        """Calculate concurrence (proper entanglement measure for 2-qubit states)"""
        # Pauli-Y matrix
        sigma_y = np.array([[0, -1j], [1j, 0]])
        
        # Spin-flip operation
        sigma_y_tensor = np.kron(sigma_y, sigma_y)
        
        # Calculate rho_tilde = (σ_y ⊗ σ_y) ρ* (σ_y ⊗ σ_y)
        rho_tilde = sigma_y_tensor @ np.conj(rho_2q) @ sigma_y_tensor
        
        # Calculate R = ρ × ρ_tilde
        R = rho_2q @ rho_tilde
        
        # Get eigenvalues and sort in descending order
        eigenvals = np.linalg.eigvals(R)
        eigenvals = np.sqrt(np.maximum(eigenvals.real, 0))  # Take real part and ensure non-negative
        eigenvals = np.sort(eigenvals)[::-1]  # Sort in descending order
        
        # Concurrence formula
        concurrence = max(0, eigenvals[0] - eigenvals[1] - eigenvals[2] - eigenvals[3])
        
        return concurrence
    
    # Part A: Define pure two-qubit states manually
    print("Pure Two-Qubit States:")
    
    # |00⟩ state
    rho_00 = np.zeros((4, 4))
    rho_00[0, 0] = 1  # Only |00⟩ component
    
    # |11⟩ state
    rho_11 = np.zeros((4, 4))
    rho_11[3, 3] = 1  # Only |11⟩ component
    
    # Bell states (maximally entangled)
    # |Φ+⟩ = (|00⟩ + |11⟩)/√2
    bell_phi_plus = 0.5 * np.array([
        [1, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 1]
    ])
    
    # |Φ-⟩ = (|00⟩ - |11⟩)/√2
    bell_phi_minus = 0.5 * np.array([
        [1, 0, 0, -1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [-1, 0, 0, 1]
    ])
    
    # |Ψ+⟩ = (|01⟩ + |10⟩)/√2
    bell_psi_plus = 0.5 * np.array([
        [0, 0, 0, 0],
        [0, 1, 1, 0],
        [0, 1, 1, 0],
        [0, 0, 0, 0]
    ])
    
    # |Ψ-⟩ = (|01⟩ - |10⟩)/√2
    bell_psi_minus = 0.5 * np.array([
        [0, 0, 0, 0],
        [0, 1, -1, 0],
        [0, -1, 1, 0],
        [0, 0, 0, 0]
    ])
    
    pure_2q_states = {
        '|00⟩': rho_00,
        '|11⟩': rho_11,
        '|Φ+⟩': bell_phi_plus,
        '|Φ-⟩': bell_phi_minus,
        '|Ψ+⟩': bell_psi_plus,
        '|Ψ-⟩': bell_psi_minus
    }
    
    pure_2q_rhos = {}
    print("State    | Purity | Reduced Purity | Entanglement | Concurrence")
    print("-" * 65)
    
    for name, rho in pure_2q_states.items():
        purity = np.trace(rho @ rho).real
        reduced_purity = calculate_partial_trace_purity(rho)
        entanglement = 1 - reduced_purity  # Simple entanglement measure
        concurrence = calculate_concurrence(rho)
        
        pure_2q_rhos[name] = rho
        print(f"{name:8} | {purity:.3f}  | {reduced_purity:.6f}     | {entanglement:.6f}   | {concurrence:.6f}")
    
    # Part B: Mixed entangled states
    print("\n" + "="*65)
    print("Mixed Two-Qubit States:")
    print("="*65)
    
    # Werner state: mixture of Bell state and maximally mixed state
    bell_state = pure_2q_rhos['|Φ+⟩']
    max_mixed = np.eye(4) / 4  # Maximally mixed 2-qubit state
    
    print("\nWerner State Analysis: ρ = p|Φ+⟩⟨Φ+| + (1-p)I/4")
    print("p     | Purity | Reduced Purity | Entanglement | Concurrence")
    print("-" * 60)
    
    p_values = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
    
    for p in p_values:
        werner_state = p * bell_state + (1-p) * max_mixed
        purity = np.trace(werner_state @ werner_state).real
        reduced_purity = calculate_partial_trace_purity(werner_state)
        entanglement = 1 - reduced_purity
        concurrence = calculate_concurrence(werner_state)
        
        print(f"{p:.1f}   | {purity:.3f}  | {reduced_purity:.6f}     | {entanglement:.6f}   | {concurrence:.6f}")
    
    # Part C: Other interesting mixed states
    print("\n" + "="*65)
    print("Other Mixed Entangled States:")
    print("="*65)
    
    # Mixture of different Bell states
    print("\nMixture of Bell States: ρ = w|Φ+⟩⟨Φ+| + (1-w)|Ψ+⟩⟨Ψ+|")
    print("w     | Purity | Reduced Purity | Entanglement | Concurrence")
    print("-" * 60)
    
    bell_phi = pure_2q_rhos['|Φ+⟩']
    bell_psi = pure_2q_rhos['|Ψ+⟩']
    
    w_values = [0.0, 0.25, 0.5, 0.75, 1.0]
    
    for w in w_values:
        mixed_bell = w * bell_phi + (1-w) * bell_psi
        purity = np.trace(mixed_bell @ mixed_bell).real
        reduced_purity = calculate_partial_trace_purity(mixed_bell)
        entanglement = 1 - reduced_purity
        concurrence = calculate_concurrence(mixed_bell)
        
        print(f"{w:.2f}  | {purity:.3f}  | {reduced_purity:.6f}     | {entanglement:.6f}   | {concurrence:.6f}")
    
    # Mixture of entangled and separable states
    print("\nMixture of Entangled and Separable: ρ = q|Φ+⟩⟨Φ+| + (1-q)|00⟩⟨00|")
    print("q     | Purity | Reduced Purity | Entanglement | Concurrence")
    print("-" * 60)
    
    separable_state = pure_2q_rhos['|00⟩']
    
    q_values = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
    
    for q in q_values:
        mixed_ent_sep = q * bell_state + (1-q) * separable_state
        purity = np.trace(mixed_ent_sep @ mixed_ent_sep).real
        reduced_purity = calculate_partial_trace_purity(mixed_ent_sep)
        entanglement = 1 - reduced_purity
        concurrence = calculate_concurrence(mixed_ent_sep)
        
        print(f"{q:.1f}   | {purity:.3f}  | {reduced_purity:.6f}     | {entanglement:.6f}   | {concurrence:.6f}")
    
    # Part D: Key insights
    print("\n" + "="*65)
    print("KEY INSIGHTS:")
    print("="*65)
    
    print("\n1. Pure States:")
    print("   - All pure states have purity = 1.000")
    print("   - Separable states (|00⟩, |11⟩) have reduced purity = 1.000 (no entanglement)")
    print("   - Bell states have reduced purity = 0.500 (maximum entanglement)")
    print("   - Concurrence = 1.000 for maximally entangled Bell states")
    
    print("\n2. Werner States:")
    print("   - Purity decreases as mixing increases (p decreases)")
    print("   - Entanglement persists even in mixed states")
    print("   - Concurrence decreases linearly with mixing parameter")
    
    print("\n3. Mixed Bell States:")
    print("   - Mixing different Bell states preserves entanglement")
    print("   - All mixtures remain maximally entangled (concurrence ≈ 1.0)")
    
    print("\n4. Entangled-Separable Mixtures:")
    print("   - Entanglement decreases as separable component increases")
    print("   - Even small amounts of entangled states contribute to entanglement")
    
    print("\n5. Entanglement Measures:")
    print("   - Reduced purity: Simple but not always accurate")
    print("   - Concurrence: Proper entanglement measure for 2-qubit states")
    print("   - Concurrence = 0: Separable state")
    print("   - Concurrence = 1: Maximally entangled state")

# Run the analysis
problem_6_entanglement_analysis()




PROBLEM 6: Two-Qubit Entanglement
----------------------------------------
Pure Two-Qubit States:
State    | Purity | Reduced Purity | Entanglement | Concurrence
-----------------------------------------------------------------
|00⟩     | 1.000  | 1.000000     | 0.000000   | 0.000000
|11⟩     | 1.000  | 1.000000     | 0.000000   | 0.000000
|Φ+⟩     | 1.000  | 0.500000     | 0.500000   | 1.000000
|Φ-⟩     | 1.000  | 0.500000     | 0.500000   | 1.000000
|Ψ+⟩     | 1.000  | 0.500000     | 0.500000   | 1.000000
|Ψ-⟩     | 1.000  | 0.500000     | 0.500000   | 1.000000

Mixed Two-Qubit States:

Werner State Analysis: ρ = p|Φ+⟩⟨Φ+| + (1-p)I/4
p     | Purity | Reduced Purity | Entanglement | Concurrence
------------------------------------------------------------
0.0   | 0.250  | 0.500000     | 0.500000   | 0.000000
0.2   | 0.280  | 0.500000     | 0.500000   | 0.000000
0.4   | 0.370  | 0.500000     | 0.500000   | 0.100000
0.6   | 0.520  | 0.500000     | 0.500000   | 0.400000
0.8   | 0.730  |

In [11]:
#Challenge Problem: Complete State Tomography
def challenge_state_tomography():
    """
    Perform quantum state tomography to distinguish pure vs mixed states
    """
    print("\n\nCHALLENGE: Quantum State Tomography")
    print("-" * 40)
    
    q = cirq.GridQubit(0, 0)
    pure_sim = Simulator()
    
    def measure_pauli_expectation(circuit, pauli_op, shots=10000):
        """Measure expectation value of Pauli operator"""
        if pauli_op == 'X':
            measure_circuit = circuit + cirq.Circuit(cirq.H(q), cirq.measure(q, key='result'))
        elif pauli_op == 'Y':
            measure_circuit = circuit + cirq.Circuit(cirq.S(q)**-1, cirq.H(q), cirq.measure(q, key='result'))
        elif pauli_op == 'Z':
            measure_circuit = circuit + cirq.Circuit(cirq.measure(q, key='result'))
        
        result = pure_sim.run(measure_circuit, repetitions=shots)
        counts = result.histogram(key='result')
        
        p0 = counts.get(0, 0) / shots
        p1 = counts.get(1, 0) / shots
        
        return p0 - p1  # Expectation value
    
    def reconstruct_density_matrix(exp_x, exp_y, exp_z):
        """Reconstruct density matrix from Pauli expectations"""
        return 0.5 * np.array([
            [1 + exp_z, exp_x - 1j * exp_y],
            [exp_x + 1j * exp_y, 1 - exp_z]
        ])
    
    # Test states
    test_circuits = {
        'Unknown State 1': cirq.Circuit(cirq.ry(np.pi/3)(q)),
        'Unknown State 2': cirq.Circuit(cirq.rx(np.pi/4)(q), cirq.rz(np.pi/6)(q))
    }
    
    print("State Tomography Results:")
    
    for name, circuit in test_circuits.items():
        print(f"\n{name}:")
        
        # Measure Pauli expectations
        exp_x = measure_pauli_expectation(circuit, 'X')
        exp_y = measure_pauli_expectation(circuit, 'Y')
        exp_z = measure_pauli_expectation(circuit, 'Z')
        
        # Reconstruct density matrix
        rho_reconstructed = reconstruct_density_matrix(exp_x, exp_y, exp_z)
        
        # Calculate properties
        purity = np.trace(rho_reconstructed @ rho_reconstructed).real
        bloch_length = np.sqrt(exp_x**2 + exp_y**2 + exp_z**2)
        
        print(f"  Pauli expectations: X={exp_x:.3f}, Y={exp_y:.3f}, Z={exp_z:.3f}")
        print(f"  Bloch vector length: {bloch_length:.3f}")
        print(f"  Purity: {purity:.3f}")
        print(f"  State type: {'Pure' if purity > 0.99 else 'Mixed'}")
        print(f"  Reconstructed ρ:\n{np.around(rho_reconstructed, 3)}")

challenge_state_tomography()




CHALLENGE: Quantum State Tomography
----------------------------------------
State Tomography Results:

Unknown State 1:
  Pauli expectations: X=0.867, Y=0.002, Z=0.504
  Bloch vector length: 1.003
  Purity: 1.003
  State type: Pure
  Reconstructed ρ:
[[0.752+0.j    0.434-0.001j]
 [0.434+0.001j 0.248+0.j   ]]

Unknown State 2:
  Pauli expectations: X=0.346, Y=-0.623, Z=0.715
  Bloch vector length: 1.010
  Purity: 1.010
  State type: Pure
  Reconstructed ρ:
[[0.858+0.j    0.173+0.312j]
 [0.173-0.312j 0.142+0.j   ]]


These problems demonstrate:

Purity as the fundamental distinguisher between pure (P=1) and mixed (P<1) states

Measurement statistics showing how mixed states average over pure state probabilities

Bloch sphere geometry where pure states are on the surface and mixed states are inside

Thermal states as important examples of mixed states in quantum thermodynamics

Decoherence processes that transform pure states into mixed states

Entanglement in both pure and mixed two-qubit systems

State tomography techniques to experimentally distinguish state types

Each problem builds understanding of the fundamental differences between pure and mixed quantum states through hands-on programming with local simulators.